In [1]:
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import relationship
from sqlalchemy import ForeignKey
import os
import logging
from datetime import datetime

In [2]:
database_file = "wiki_data.db"
engine = create_engine(f"sqlite:///{database_file}")    

In [3]:
import pandas as pd
from sqlalchemy import inspect

# Create the inspector
inspector = inspect(engine)

# Get the table names
table_names = inspector.get_table_names()

In [4]:
lista_dfs = []
# Iterate over the table names
for table_name in table_names:
    # Load the table content into a DataFrame
    df = pd.read_sql_table(table_name, engine)
    
    # Print the table name
    print(f"Table: {table_name}")
    
    lista_dfs.append(df)

Table: facts
Table: featured_images
Table: preview_links


In [5]:
i = 0
print(table_names[i])
lista_dfs[i]

facts


,id,content,created_at,day_of_week,analyzed,internal_links,campaign_parameters
0,1,... that before Major League Baseball pitcher ...,2023-07-08 20:56:29.982419,Saturday,True,"'https://en.wikipedia.org/wiki/2023', 'https:/...","{'date': None, 'length': 28}"
1,2,... that Piri's self-produced music video for ...,2023-07-08 20:56:29.986407,Saturday,True,"'https://en.wikipedia.org/wiki/Bedroom', 'http...",{'length': 25}
2,3,... that the 17-animal inheritance puzzle has ...,2023-07-08 20:56:29.987694,Saturday,True,"'https://en.wikipedia.org/wiki/17', 'https://e...",{'length': 18}
3,4,... that Seda Kaçan became Turkey's first race...,2023-07-08 20:56:29.988751,Saturday,True,"'https://en.wikipedia.org/wiki/Driver', 'https...",{'length': 10}
4,5,... that the Vegas Golden Knights used five di...,2023-07-08 20:56:29.989820,Saturday,True,'https://en.wikipedia.org/wiki/2022–23_UEFA_Ch...,{'length': 18}
5,6,... that Leonard Abrams founded the arts and c...,2023-07-08 20:56:29.990880,Saturday,True,"'https://en.wikipedia.org/wiki/Abrams', 'https...",{'length': 27}
6,7,... that though the Offerman Building is only ...,2023-07-08 20:56:29.991941,Saturday,True,"'https://en.wikipedia.org/wiki/Brooklyn', 'htt...",{'length': 24}
7,8,... that John Oliver dared Disney to sue him o...,2023-07-08 20:56:29.993002,Saturday,True,"'https://en.wikipedia.org/wiki/Dare', 'https:/...",{'length': 15}


In [6]:
lista_dfs[i]['content'][0]

'... that before Major League Baseball pitcher Keaton Winn (pictured) was called up to the majors in June 2023, he had never been to a major league stadium?'

In [7]:
print(lista_dfs[i]['internal_links'][0][1:-1])

https://en.wikipedia.org/wiki/2023', 'https://en.wikipedia.org/wiki/2023_Cricket_World_Cup', "https://en.wikipedia.org/wiki/2023_FIFA_Women's_World_Cup", 'https://en.wikipedia.org/wiki/2023_NBA_draft', 'https://en.wikipedia.org/wiki/2023_Turkey–Syria_earthquake', 'https://en.wikipedia.org/wiki/Baseball', 'https://en.wikipedia.org/wiki/Major_League_Baseball', 'https://en.wikipedia.org/wiki/List_of_baseball_video_games', 'https://en.wikipedia.org/wiki/Triple-A_(baseball)', 'https://en.wikipedia.org/wiki/Perfect_game_(baseball)', 'https://en.wikipedia.org/wiki/Call', 'https://en.wikipedia.org/wiki/Computer-assisted_language_learning', 'https://en.wikipedia.org/wiki/On-call', 'https://en.wikipedia.org/wiki/The_Call', 'https://en.wikipedia.org/wiki/Call_of_Duty', 'https://en.wikipedia.org/wiki/June', 'https://en.wikipedia.org/wiki/June_30', 'https://en.wikipedia.org/wiki/June_26', 'https://en.wikipedia.org/wiki/June_25', 'https://en.wikipedia.org/wiki/June_29', 'https://en.wikipedia.org/wik

In [8]:
lista_dfs[i]['campaign_parameters'][0]

"{'date': None, 'length': 28}"

In [9]:
i = 1
print(table_names[i])
lista_dfs[i]

featured_images


,id,image_url,caption,fact_id
0,1,https://en.wikipedia.org/wiki/File:Win_profile...,Keaton Winn,1


In [10]:
i = 2
print(table_names[i])
lista_dfs[i]

preview_links


,id,url,fact_id
0,1,https://en.wikipedia.org/wiki/Major_League_Bas...,1
1,2,https://en.wikipedia.org/wiki/Keaton_Winn,1
2,3,https://en.wikipedia.org/wiki/Piri,2
3,4,https://en.wikipedia.org/wiki/Soft_Spot_(song),2
4,5,https://en.wikipedia.org/wiki/Projector,2
5,6,https://en.wikipedia.org/wiki/YouTube,2
6,7,https://en.wikipedia.org/wiki/17-animal_inheri...,3
7,8,https://en.wikipedia.org/wiki/Seda_Ka%C3%A7an,4
8,9,https://en.wikipedia.org/wiki/Vegas_Golden_Kni...,5
9,10,https://en.wikipedia.org/wiki/2022%E2%80%9323_...,5


In [90]:
query = '''
    SELECT f.*, fi.*, GROUP_CONCAT(DISTINCT pl.id) AS pl_id,  GROUP_CONCAT(DISTINCT pl.url) AS preview_urls, pl.fact_id AS fact_id_2
    FROM facts f
    LEFT JOIN featured_images fi ON fi.fact_id = f.id
    LEFT JOIN preview_links pl ON pl.fact_id = f.id
    --WHERE f.id = 1
    GROUP BY f.id
    '''
query_df = pd.read_sql_query(query, engine)
query_df.columns = ['id', 'content', 'created_at', 'day_of_week', 'analyzed',
                    'internal_links', 'campaign_parameters', 'fi_id', 'image_url', 'caption',
                        'fact_id_1', 'pl_id', 'preview_urls', 'fact_id_2']
query_df

,id,content,created_at,day_of_week,analyzed,internal_links,campaign_parameters,fi_id,image_url,caption,fact_id_1,pl_id,preview_urls,fact_id_2
0,1,... that before Major League Baseball pitcher ...,2023-07-08 20:56:29.982419,Saturday,1,"'https://en.wikipedia.org/wiki/2023', 'https:/...","{'date': None, 'length': 28}",1.0,https://en.wikipedia.org/wiki/File:Win_profile...,Keaton Winn,1.0,"2,1","https://en.wikipedia.org/wiki/Keaton_Winn,http...",1
1,2,... that Piri's self-produced music video for ...,2023-07-08 20:56:29.986407,Saturday,1,"'https://en.wikipedia.org/wiki/Bedroom', 'http...",{'length': 25},NaN,None,None,NaN,"3,5,4,6","https://en.wikipedia.org/wiki/Piri,https://en....",2
2,3,... that the 17-animal inheritance puzzle has ...,2023-07-08 20:56:29.987694,Saturday,1,"'https://en.wikipedia.org/wiki/17', 'https://e...",{'length': 18},NaN,None,None,NaN,7,https://en.wikipedia.org/wiki/17-animal_inheri...,3
3,4,... that Seda Kaçan became Turkey's first race...,2023-07-08 20:56:29.988751,Saturday,1,"'https://en.wikipedia.org/wiki/Driver', 'https...",{'length': 10},NaN,None,None,NaN,8,https://en.wikipedia.org/wiki/Seda_Ka%C3%A7an,4
4,5,... that the Vegas Golden Knights used five di...,2023-07-08 20:56:29.989820,Saturday,1,'https://en.wikipedia.org/wiki/2022–23_UEFA_Ch...,{'length': 18},NaN,None,None,NaN,"10,11,9",https://en.wikipedia.org/wiki/2022%E2%80%9323_...,5
5,6,... that Leonard Abrams founded the arts and c...,2023-07-08 20:56:29.990880,Saturday,1,"'https://en.wikipedia.org/wiki/Abrams', 'https...",{'length': 27},NaN,None,None,NaN,"13,14,12",https://en.wikipedia.org/wiki/East_Village_Eye...,6
6,7,... that though the Offerman Building is only ...,2023-07-08 20:56:29.991941,Saturday,1,"'https://en.wikipedia.org/wiki/Brooklyn', 'htt...",{'length': 24},NaN,None,None,NaN,"16,15",https://en.wikipedia.org/wiki/List_of_tallest_...,7
7,8,... that John Oliver dared Disney to sue him o...,2023-07-08 20:56:29.993002,Saturday,1,"'https://en.wikipedia.org/wiki/Dare', 'https:/...",{'length': 15},NaN,None,None,NaN,"17,18","https://en.wikipedia.org/wiki/John_Oliver,http...",8


In [91]:
query_df

,id,content,created_at,day_of_week,analyzed,internal_links,campaign_parameters,fi_id,image_url,caption,fact_id_1,pl_id,preview_urls,fact_id_2
0,1,... that before Major League Baseball pitcher ...,2023-07-08 20:56:29.982419,Saturday,1,"'https://en.wikipedia.org/wiki/2023', 'https:/...","{'date': None, 'length': 28}",1.0,https://en.wikipedia.org/wiki/File:Win_profile...,Keaton Winn,1.0,"2,1","https://en.wikipedia.org/wiki/Keaton_Winn,http...",1
1,2,... that Piri's self-produced music video for ...,2023-07-08 20:56:29.986407,Saturday,1,"'https://en.wikipedia.org/wiki/Bedroom', 'http...",{'length': 25},NaN,None,None,NaN,"3,5,4,6","https://en.wikipedia.org/wiki/Piri,https://en....",2
2,3,... that the 17-animal inheritance puzzle has ...,2023-07-08 20:56:29.987694,Saturday,1,"'https://en.wikipedia.org/wiki/17', 'https://e...",{'length': 18},NaN,None,None,NaN,7,https://en.wikipedia.org/wiki/17-animal_inheri...,3
3,4,... that Seda Kaçan became Turkey's first race...,2023-07-08 20:56:29.988751,Saturday,1,"'https://en.wikipedia.org/wiki/Driver', 'https...",{'length': 10},NaN,None,None,NaN,8,https://en.wikipedia.org/wiki/Seda_Ka%C3%A7an,4
4,5,... that the Vegas Golden Knights used five di...,2023-07-08 20:56:29.989820,Saturday,1,'https://en.wikipedia.org/wiki/2022–23_UEFA_Ch...,{'length': 18},NaN,None,None,NaN,"10,11,9",https://en.wikipedia.org/wiki/2022%E2%80%9323_...,5
5,6,... that Leonard Abrams founded the arts and c...,2023-07-08 20:56:29.990880,Saturday,1,"'https://en.wikipedia.org/wiki/Abrams', 'https...",{'length': 27},NaN,None,None,NaN,"13,14,12",https://en.wikipedia.org/wiki/East_Village_Eye...,6
6,7,... that though the Offerman Building is only ...,2023-07-08 20:56:29.991941,Saturday,1,"'https://en.wikipedia.org/wiki/Brooklyn', 'htt...",{'length': 24},NaN,None,None,NaN,"16,15",https://en.wikipedia.org/wiki/List_of_tallest_...,7
7,8,... that John Oliver dared Disney to sue him o...,2023-07-08 20:56:29.993002,Saturday,1,"'https://en.wikipedia.org/wiki/Dare', 'https:/...",{'length': 15},NaN,None,None,NaN,"17,18","https://en.wikipedia.org/wiki/John_Oliver,http...",8


In [92]:
query_df[['pl_id','preview_urls','fact_id_2']]['preview_urls']

0    https://en.wikipedia.org/wiki/Keaton_Winn,http...
1    https://en.wikipedia.org/wiki/Piri,https://en....
2    https://en.wikipedia.org/wiki/17-animal_inheri...
3        https://en.wikipedia.org/wiki/Seda_Ka%C3%A7an
4    https://en.wikipedia.org/wiki/2022%E2%80%9323_...
5    https://en.wikipedia.org/wiki/East_Village_Eye...
6    https://en.wikipedia.org/wiki/List_of_tallest_...
7    https://en.wikipedia.org/wiki/John_Oliver,http...
Name: preview_urls, dtype: object

In [89]:
query_df[['pl_id','preview_urls','fact_id_2']]['preview_urls'][0].split(',')

['https://en.wikipedia.org/wiki/Keaton_Winn',
 'https://en.wikipedia.org/wiki/Major_League_Baseball']